<p style="color: red; font-size: 16pt; font-weight: bold; text-align:center;">Change the name of this notebook before you edit!</p>

# Motor Insurance Fraud data from textbook

# Setup

In [1]:
%reload_ext autoreload
%autoreload 2

import sys
import os
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Load Data

In [2]:
! ls /data/kelleher/*

/data/kelleher/MotorInsuranceFraudClaimABT_clean.csv
/data/kelleher/MotorInsuranceFraudClaimABTFull.csv
/data/kelleher/MotorInsuranceFraudClaim_ClaimAmount.csv
/data/kelleher/MotorInsuranceFraudClaim_ClaimAmountReceived.csv
/data/kelleher/MotorInsuranceFraudClaim_Claimed_vs_Received.csv
/data/kelleher/MotorInsuranceFraudClaim_clean.csv
/data/kelleher/MotorInsuranceFraudClaim_IncomeofPolicyHolder.csv
/data/kelleher/MotorInsuranceFraudClaim_InsuranceType.csv
/data/kelleher/MotorInsuranceFraudClaim_MaritalStatus.csv
/data/kelleher/MotorInsuranceFraudClaim_NumClaims.csv
/data/kelleher/MotorInsuranceFraudClaim_NumSoftTissue.csv
/data/kelleher/MotorInsuranceFraudClaim_PercentSoftTissue.csv
/data/kelleher/MotorInsuranceFraudClaim_TotalClaimed.csv


In [3]:
_messy_df = pd.read_csv('/data/kelleher/MotorInsuranceFraudClaimABTFull.csv')
for f,t in _messy_df.dtypes.items():
    print(f"`{f}`: {t}")

`ID`: int64
`Insurance Type `: object
`Income of Policy Holder`: int64
`Marital Status`: object
`Num Claimants`: int64
`Injury Type`: object
`Overnight Hospital Stay`: object
`Claim Amount`: int64
`Total Claimed`: int64
`Num Claims`: int64
`Num Soft Tissue`: float64
`% Soft Tissue`: float64
`Claim Amount Received`: int64
`Fraud Flag`: int64


We don't like spaces and special characters in column names. One column has a space character at the end...that's easy to overlook.

In [4]:
claim_df = pd.read_csv('/data/kelleher/MotorInsuranceFraudClaimABT_clean.csv')
print(f"Number of records: {claim_df.shape[0]:,}")
display(claim_df.dtypes)

Number of records: 500


ID                         int64
InsuranceType             object
IncomeofPolicyHolder       int64
MaritalStatus             object
NumClaimants               int64
InjuryType                object
OvernightHospitalStay     object
ClaimAmount                int64
TotalClaimed               int64
NumClaims                  int64
NumSoftTissue            float64
PercentSoftTissue        float64
ClaimAmountReceived        int64
FraudFlag                  int64
dtype: object

In [5]:
claim_df.head(3).T

,0,1,2
ID,1,2,3
InsuranceType,CI,CI,CI
IncomeofPolicyHolder,0,0,54613
MaritalStatus,NaN,NaN,Married
NumClaimants,2,2,1
InjuryType,Soft Tissue,Back,Broken Limb
OvernightHospitalStay,No,Yes,No
ClaimAmount,1625,15028,-99999
TotalClaimed,3250,60112,0
NumClaims,2,1,0


In [6]:
claim_df.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
ID,500.0,NaN,NaN,NaN,250.5,144.481833,1.0,125.75,250.5,375.25,500.0
InsuranceType,500,1,CI,500,NaN,NaN,NaN,NaN,NaN,NaN,NaN
IncomeofPolicyHolder,500.0,NaN,NaN,NaN,13739.994,20081.535489,0.0,0.0,0.0,33918.5,71284.0
MaritalStatus,170,3,Married,99,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NumClaimants,500.0,NaN,NaN,NaN,1.908,1.012713,1.0,1.0,2.0,3.0,4.0
InjuryType,500,4,Broken Limb,177,NaN,NaN,NaN,NaN,NaN,NaN,NaN
OvernightHospitalStay,500,2,No,354,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ClaimAmount,500.0,NaN,NaN,NaN,16373.204,29426.27696,-99999.0,3322.25,5663.0,12245.5,270200.0
TotalClaimed,500.0,NaN,NaN,NaN,9597.186,35655.68622,0.0,0.0,0.0,11282.75,729792.0
NumClaims,500.0,NaN,NaN,NaN,0.798,2.666724,0.0,0.0,0.0,1.0,56.0


In [7]:
claim_df[['ClaimAmount', 'TotalClaimed', 'NumClaims', 'ClaimAmountReceived']].to_numpy()

array([[  1625,   3250,      2,      0],
       [ 15028,  60112,      1,  15028],
       [-99999,      0,      0,    572],
       ...,
       [ 32469,      0,      0,  16763],
       [179448,      0,      0, 179448],
       [  8259,      0,      0,      0]])